# BackPropagation算法的矩阵形式推导

## 符号说明
|符号|说明|
|:|:|
|$l$|神经网络的第$l$层，$1\leq l\leq L$|
|$d^{(l)}$|第$l$层的节点个数|
| $W^{(l)}$|$d^{(l)}\times d^{(l-1)}$的权值矩阵，$W^{(l)}_{ij}$表示第$l-1$层节点$j$到第$l$层节点$i$的突触权值|
| $x^{(l)}$|第$l$层每个节点输出组成的向量，是一个$d^{(l)}\times 1$的列向量|
|$s^{(l)}$|第$l$层每个节点输入组成的向量，同上|
|$b^{(l)}$|第$l$层每个节点的偏置，同上|
| $\sigma(x)$|_sigmoid_函数，$\sigma(x)=1/(1+exp(-x))$，它的导数是$\sigma^\prime(x)=\sigma(x)(1-\sigma(x))$|
| $n$|一个mini-batch上的样本数|
|$y_i$|第i个样本的标签，为one-hot向量|
|$\hat{y}_i$|第i个样本的softmax输出，其维度与$y_i$相同|
| $e_i$|当输入为$(x_i,y_i)$时的损失函数|
|$\delta^{(l)}$|第$l$层的敏感度，定义为$\delta^{(l)}={\partial e}/{\partial s^{(l)}}$|

## 前馈过程和目标函数
神经网络的前馈公式：
$$ \left\{ \begin{aligned} & s^{(l)} = W^{(l)} \cdot x^{(l-1)}  + b^{(l)}\\ & x^{(l)} = \sigma(s^{(l)})  \end{aligned} \right. $$
其中$\sigma(\cdot)$是sigmoid或softmax函数。  
一般对于多分类的场合，神经网络的目标是最小化交叉熵误差。特别地，对于样本$(x_i,y_i)$来说：
$$ e_i =  -y_i^T \log(\hat{y}_i)$$

通常为了加快神经网络的运算速度，我们采用mini-batch sgd的方式来更新权值，mini-batch版本的损失函数为：
$$ E =  \frac{1}{n} \sum_{i=1}^n e_i=-\frac{1}{n} \sum_{i=1}^n y_i^T \log(\hat{y}_i)$$
其中，$n$为mini-batch中的样本数，$\hat{y}_i=softmax(W^{(L)}x_i^{(L-1)}+ b^{(L)})$是神经网络的输出

## BP算法推导
BP算法的关键是计算$\delta^{(l)}=\frac{\partial E}{\partial s^{(l)}}$，其含义是最终误差对于第$l$层神经元的敏感程度。  
### 1.输出层的敏感度$\frac{\partial E}{\partial s^{(L)}}$  
首先列出各变量间的关系
$$\left\{\begin{aligned}&s^{(L)}=W^{(L)}x^{(L-1)}+b^{(L)}\\& \hat{y}=\mathrm{softmax}(s^{(L)})=\frac{exp(s^{(L)})}{1_c^Texp(s^{(L)})} \end{aligned}\right.$$
依据chain rule有：
$$ \begin{aligned}\frac{\partial E}{\partial s^{(L)}}
&=\frac {\partial \hat{y}}{\partial s^{(L)}}\frac{\partial E}{\partial \hat{y}}\end{aligned}$$
注意上式的相乘顺序，这里我用了`Denominator layout`，因此套用链式法则时要颠倒相乘顺序，原先是
$$E\to \hat{y}, \hat{y} \to s^{(L)}$$
用了`Denominator layout`后要反过来，变为了
$$\hat{y} \to s^{(L)} , E\to \hat{y}$$
1)$\frac{\partial \hat{y}}{\partial s^{(L)}}$  
$$\begin{equation}\begin{aligned}\frac{\partial \hat{y}}{\partial s^{(L)}}&=\frac{\partial ( \frac{exp(s^{(L)})}{1_c^Texp(s^{(L)})})}{\partial s^{(L)}}\\&= \frac{1}{1_c^Texp(s^{(L)})}\frac{\partial exp(s^{(L)})}{\partial s^{(L)}}+ \frac{\partial (\frac{1}{1_c^Texp(s^{(L)})})}{\partial s^{(L)}}( exp(s^{(L)}) )^T\\&= \frac{1}{1_c^Texp(s^{(L)})}diag(exp(s^{(L)}))-\frac{1}{(1_c^Texp(s^{(L)}))^2}exp(s^{(L)})exp(s^{(L)})^T\\&=diag(\frac{exp(s^{(L)})}{1_c^Texp(s^{(L)})})-\frac{exp(s^{(L)})}{1_c^Texp(s^{(L)})}\cdot (\frac{exp(s^{(L)})}{1_c^Texp(s^{(L)})})^T\\&=diag(\mathrm{ softmax}(s^{(L)}))- \mathrm{ softmax}(s^{(L)}) \mathrm{ softmax}(s^{(L)})^T\\&=diag(\hat{y})-\hat{y}\hat{y}^T \end{aligned}\end{equation}$$
2)$\frac{\partial E}{\partial \hat{y}}$  
同样应用反向链式法则有：
$$\begin{equation}\frac{\partial E}{\partial \hat{y}}=\frac{\partial log(\hat{y})}{\partial \hat{y}}\cdot \frac{\partial (-y^Tlog(\hat{y}))}{\partial log(\hat{y})}=\big(diag(\hat{y})\big)^{-1}\cdot(-y)\end{equation}$$

综合以上，有
$$ \begin{aligned}\frac{\partial E}{\partial s^{(L)}}
&=\frac {\partial \hat{y}}{\partial s^{(L)}}\frac{\partial E}{\partial \hat{y}}\\&=\bigg(diag(\hat{y})-\hat{y}\hat{y}^T\bigg) \big(diag(\hat{y})\big)^{-1}\cdot(-y)\\&=-y+\hat{y}(\mathbf{1}^Ty)\\&=\hat{y}-y\end{aligned}$$

其中$\hat{y}^T\cdot diag(\hat{y})^{-1}=\mathbf{1}^T=(1,1,...,1)^T$表示一个全为1的向量，因为$y$是一个one hot向量，因此$\mathbf{1}^Ty=\sum_{i=1}^n y_i=1$  
因此我们知道：
>$$\delta^{(L)}=\frac{\partial E}{\partial s^{(L)}}=\hat{y}-y$$



### 2.隐层的敏感度
依据链式法则，有  
$$ \begin{aligned}\delta^{(l)}=\frac{\partial E}{\partial s^{(l)}} = &( \frac{\partial E}{\partial s^{(l+1)}} \cdot \frac{\partial s^{(l+1)}}{\partial x^{(l)}} \cdot\frac{\partial x^{(l)}}{\partial s^{(l)}} ) ^T\\ = &\{ (\delta^{(l+1)})^T \cdot W^{(l+1)} \cdot diag(\sigma^\prime(s^{(l)}))\}^T\\=& diag(\sigma^\prime(s^{(l)}))\cdot (W^{(l+1)})^T\delta^{(l+1)}\\=& \sigma^\prime(s^{(l)})\circ ((W^{(l+1)})^T\delta^{(l+1)})\end{aligned}$$
其中$\circ$是向量点积（或Hadamard product，element-wise product）。于是我们有了误差敏感度的递推公式：

>$$ \begin{aligned}\delta^{(l)} &= \sigma^\prime(s^{(l)})\circ ((W^{(l+1)})^T\delta^{(l+1)})\\&= \sigma(s^{(l)})\circ (1-\sigma(s^{(l)}))\circ ((W^{(l+1)})^T\delta^{(l+1)})\end{aligned}$$

我们发现，敏感度满足一个递推式，当前层的敏感度依赖于下一层的敏感度。在进行反向传播时，首先我们需要先计算最后一层（即输出层）的敏感度$\delta^{(L)}$，再回传给$L-1$层，计算$\delta^{(L-1)}$，接着按照从后向前的顺序依次遍历剩余所有层，计算$\delta^{(l)}$，这个过程可以表示为：
$$\delta^{(L)}\to \delta^{(L-1)}\to \delta^{(L-2)}\to\cdots\delta^{(1)}$$

## mini-batch sgd
前面的推导均是基于单样本的sgd，当以一个mini-batch作为输入时，梯度应该为这个batch上的样本的梯度的平均值，即
$$ \nabla{W}^{(l)}=\frac{\partial E}{\partial W^{(l)}}=\frac{1}{n}\sum_{i=1}^n \delta_i^{(l)}\otimes x_i^{(l-1)}$$
$$\nabla{b}^{(l)}=\frac{\partial E}{\partial b^{(l)}} = \frac{1}{n}\sum_{i=1}^n \delta_i^{(l)}$$
其中$\otimes$表示向量外积  
梯度下降的更新式为：
$$ W^{(l)}\longleftarrow W^{(l)} - \lambda  \nabla{W}^{(l)}$$
$$b^{(l)}\longleftarrow b^{(l)} - \lambda\nabla{b}^{(l)}$$
其中$\lambda>0$为学习率